In [3]:
import random as r

from engine import Value
from modules import *
from ops import *
from gpt import *

In [5]:
batch_size = 2
vocab_len = 10
model_dim = 8
max_seq_len = 5
seq_len = 3
num_heads = 2
head_dim = 4
mlp_mult = 4

In [ ]:
class CrossEntropyLoss(Module):
    def __init__(self, pad_token):
        self.pad_token = pad_token

    def __call__(self, logits, targets):
        '''
        inputs: 
        logits - list of lists of lists of shape (batch_size, seq_len, vocab_len) full of Value objects
        targets - list of lists of shape (batch_size, seq_len) full of integers representing token indices

        output: a single Value object representing loss of the model
        '''

In [ ]:
# TODO: make Embedding module also do unembedding w/ shared weights? 
# wouldn't be exactly faithful to pytorch implementation but i'd like to use gradient accumulation & save parameters

In [14]:
x = [[[Value(r.uniform(-1,1)) for _ in range(model_dim)]
      for _ in range(seq_len)]
     for _ in range(batch_size)]
print(get_shape(x))
layer = ResidualLayer(model_dim, num_heads, head_dim, max_seq_len, mlp_mult)
y = layer(x)
print(get_shape(y))

[2, 3, 8]
[2, 3, 8]
